this notebook consists the following sessions:
* catalog sensation (check this Notion page: https://www.notion.so/Catalog-Item-API-c57684b509504f2cb3faa70eb5a66fb4)
* spread sheet as input-resource
* workflow
* Q&A

In [1]:
# ref
# Design a Catalog: https://developer.squareup.com/docs/catalog-api/design-a-catalog

In [2]:
## Add this to the first block in your note book to display json
# another option: 
import uuid
from IPython.core.display import display, HTML

import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())
        # This line is missed out in most of the versions of this script across the web, it is essential for this to work interleaved with print statements
        self._ipython_display_()
        
    def _ipython_display_(self):
        display(HTML('<div id="{}" style="height: auto; width:100%;"></div>'.format(self.uuid)))
        display(HTML("""<script>
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });</script>
        """ % (self.uuid, self.json_str)))

# Since this is copy-pasted wrongly(mostly) at a lot of places across the web, i'm putting the fixed, updated version here, mainly for self-reference


## To use this function, call this, this now works even when you have a print statement before or after the RenderJSON call
# RenderJSON(your_json)
# RenderJSON(json.loads(result.text))

In [3]:
# catalog sensation (check this Notion page: https://www.notion.so/Catalog-Item-API-c57684b509504f2cb3faa70eb5a66fb4)

In [4]:
# ...

In [5]:
# spread sheet as input-resource

In [6]:
import pandas as pd

In [7]:
# Note: the XX_id col are NAN, we will fill it with some string for now
# but the REAL id will be retrived from the result proxy (see later)

# Note: if variation_name left empty(i.g., the item only has one variation of itself), 
# Square API will fill variation_name as default == "Regular" (here: we fillna with "Default")

df_original = pd.read_excel("/home/hadoop/Desktop/SharedFolder/item_input_0605.xlsx")
df = df_original.copy()
df[["variation_id","item_id","category_id"]] = df[["variation_id","item_id","category_id"]].fillna("NANNAN")
df[["variation_name","item_description"]] = df[["variation_name","item_description"]].fillna("Default")
# df["variation_id"] = df["variation_id"].apply(lambda x: str(uuid.uuid4()))
df

,variation_id,variation_name,item_id,item_name,category_id,category_name,variation_amount,variation_currency,item_description
0,NANNAN,Small red,NANNAN,Shirt,NANNAN,Clothes,2500,USD,this is a shirt item
1,NANNAN,Small blue,NANNAN,Shirt,NANNAN,Clothes,2500,USD,this is a shirt item
2,NANNAN,Large red,NANNAN,Shirt,NANNAN,Clothes,3000,USD,this is a shirt item
3,NANNAN,Large blue,NANNAN,Shirt,NANNAN,Clothes,3000,USD,this is a shirt item
4,NANNAN,Blue,NANNAN,Hat,NANNAN,Accessories,1000,USD,this is a hat item
5,NANNAN,Red,NANNAN,Hat,NANNAN,Accessories,1000,USD,this is a hat item
6,NANNAN,Green,NANNAN,Scarf,NANNAN,Accessories,700,USD,this is a scarf item
7,NANNAN,Red,NANNAN,Scarf,NANNAN,Accessories,700,USD,this is a scarf item
8,NANNAN,Default,NANNAN,Sunglasses,NANNAN,Accessories,1500,USD,Default


Workflow
* make connection
* use Upsert catalog (part of catalog sub-API) to insert objects from up to down
    * 1st layer: category
    * 2nd layer: item
    * 3rd layer: variation


In [10]:
# module
# sandbox connection 

from square.client import Client
sandbox_token = "EAAAELeY2Gk_kgDuX7SFsmt-d3UzMTsdB9nRf2ew8EjO1wDdnpI1LwOV0KobTx4g" # this is invalid
# actually "don't hard code the token, instead save as file"
with open('/home/hadoop/square_sandbox_token', 'r') as reader:
    sandbox_token = sandbox_token = reader.read().splitlines()[0]

client = Client(
    access_token = sandbox_token,
    environment='sandbox',
)

# test connection
result = client.catalog.list_catalog(
  types = "ITEM"
)
result.status_code


In [11]:
# part III: upsert catalog object
# upsert = update + insert

In [12]:
# start with category

In [33]:
################## to improve #####################
# note 1: the API has some capacity when inserting data,(100 object at a time using batch upsert)
# since the item library is not crazily huge. 
# we can insert one row at a time

# note 2: the way Square handle id is a bit bizzare, 
# the "id" we feed in right now is just a reference,
# we will retrieve the actual object id from the result proxy

# note 3: the REAL id is only identifier SquareUp uses. So, it is very likely there exist "different"
# objects with the same names.


def insert_category(row):
    result = client.catalog.upsert_catalog_object(
    body = {
        "idempotency_key": str(uuid.uuid1()),
        "object": {
            "type": "CATEGORY",
            "id": "#"+row["category_name"], ## this is not the real id, but some reference
            "category_data": {
                "name": row["category_name"]
            }}})
    return result
    
df_category = df[["category_id", "category_name"]].groupby("category_name").first().reset_index()
# df_category
    
# df_category.apply(lambda x: print(x.category_id), axis = 1)
df_category["result"] = df_category.apply(lambda x: insert_category(x), axis = 1)
df_category["category_id"] = df_category.result.apply(lambda x: x.body.get("catalog_object").get("id"))
df_category

,category_name,category_id,result
0,Accessories,G7MKUFJHREIIZWQG4AI4FUCT,"<ApiResponse [{""catalog_object"":{""type"":""CATEG..."
1,Clothes,HMDBGTNWIZGZF2YU7NZJZO4X,"<ApiResponse [{""catalog_object"":{""type"":""CATEG..."


In [14]:
# review catalogs
# ref: Catalog -> List catalog. https://developer.squareup.com/explorer/square/catalog-api/list-catalog

result = client.catalog.list_catalog(
  types = "CATEGORY"
)

RenderJSON(result.body)

In [35]:
# merge back (use left_outer explicitly to reserve the order)
df = pd.merge(df.drop("category_id", axis=1), df_category.drop("result", axis=1), on="category_name", how= 'left')

In [16]:
# some cleaning work (if needed): 
# Sometimes we might want to delete the whole library and start from scratch
# EXAMPLE delete all the categories
# step 1: retrive object IDs
# step 2: delete object by using IDs

# Note: I am presenting a simplified version of list catalog. We will talk about
# a more "proper" way to list objects later

def clear_catalog(types):
#     types: "CATEGORY" or "ITEM", "ITEM_VARIATION"
    # step 1
    # ref: Catalog -> List catalog. https://developer.squareup.com/explorer/square/catalog-api/list-catalog
    result = client.catalog.list_catalog(
      types = types
    )
    # retrive the ids
    ids_to_del = pd.json_normalize(result.body.get("objects")).id.values

    #step 2
    # ref: Catalog -> Delete catalog object. https://developer.squareup.com/explorer/square/catalog-api/delete-catalog-object
    for obj in ids_to_del:
        result = client.catalog.delete_catalog_object(
        object_id = obj
        )
        
# clear_catalog("CATEGORY")
# clear_catalog("ITEM_VARIATION")
# if types=None, all the catalogs will be deleted
# clear_catalog(None)

In [17]:
# stop here on purpose: take a look at the dashboard: categories

In [18]:
# 2nd layer: insert items with category created earlier

In [36]:
def insert_item(row):
    result = client.catalog.upsert_catalog_object(
  body = {
    "idempotency_key": str(uuid.uuid4()),
    "object": {
      "type": "ITEM",
      "id": "#"+row["item_name"],
      "item_data": {
        "name": row["item_name"],
        "description": row["item_description"],
        "category_id": row["category_id"]
      }
    }
  }
)
    return result
    
df_item = df[["item_id", "item_name", "item_description", "category_id"]].groupby("item_name").first().reset_index()
# df_category
    
# df_category.apply(lambda x: print(x.category_id), axis = 1)
df_item["result"] = df_item.apply(lambda x: insert_item(x), axis = 1)
df_item["item_id"] = df_item.result.apply(lambda x: x.body.get("catalog_object").get("id"))
df_item

,item_name,item_id,item_description,category_id,result
0,Hat,2S33CD4247XXEORTAT5XYOQB,this is a hat item,G7MKUFJHREIIZWQG4AI4FUCT,"<ApiResponse [{""catalog_object"":{""type"":""ITEM""..."
1,Scarf,THBCUS7E6THY4UVPKZJU7VSJ,this is a scarf item,G7MKUFJHREIIZWQG4AI4FUCT,"<ApiResponse [{""catalog_object"":{""type"":""ITEM""..."
2,Shirt,JQKHXVTRQQUJC4OWF5F6HQFK,this is a shirt item,HMDBGTNWIZGZF2YU7NZJZO4X,"<ApiResponse [{""catalog_object"":{""type"":""ITEM""..."
3,Sunglasses,Z63NWS6BEBKNPDSR6AXPUSYV,Default,G7MKUFJHREIIZWQG4AI4FUCT,"<ApiResponse [{""catalog_object"":{""type"":""ITEM""..."


In [37]:
# merge back
df = pd.merge(df.drop("item_id", axis=1), 
              df_item.drop(["result", "item_description", "category_id"], axis=1), on="item_name", how= 'left')

In [21]:
# review catalogs
# ref: Catalog -> List catalog. https://developer.squareup.com/explorer/square/catalog-api/list-catalog

# Note: By default, Square products assign the item variation name "Regular", 
# so, at this stage, Square creates a "Regular" variation for each item (we will see later)

result = client.catalog.list_catalog(
  types = "ITEM"
)

RenderJSON(result.body)

In [22]:
# stop here on purpose: take a look at the dashboard: items

In [23]:
# 3rd layer: insert variations associated with items and categories created earlier

In [24]:
# clear_catalog("ITEM_VARIATION")
# clear_catalog(None)

In [38]:
def insert_variation(row):
    result = client.catalog.upsert_catalog_object(
  body = {
    "idempotency_key": str(uuid.uuid4()),
    "object": {
      "type": "ITEM_VARIATION",
      "id": "#"+row["variation_name"], #
      "item_variation_data": {
        "item_id": row["item_id"],
        "name": row["variation_name"],
        "pricing_type": "FIXED_PRICING",
        "price_money": {
          "amount": row["variation_amount"],
          "currency": row["variation_currency"]
        }
      }
    }
  }
)
    return result
    
df_variation = df[["variation_id", "variation_name", "item_id", "category_id",
                   "variation_amount", "variation_currency", ]].copy()
# df_category
    
# df_category.apply(lambda x: print(x.category_id), axis = 1)
df_variation["result"] = df_variation.apply(lambda x: insert_variation(x), axis = 1)
df_variation["variation_id"] = df_variation.result.apply(lambda x: x.body.get("catalog_object").get("id"))
df_variation

,variation_id,variation_name,item_id,category_id,variation_amount,variation_currency,result
0,J3VQOGJD4WETWRZIGJV4Z6P7,Small red,JQKHXVTRQQUJC4OWF5F6HQFK,HMDBGTNWIZGZF2YU7NZJZO4X,2500,USD,"<ApiResponse [{""catalog_object"":{""type"":""ITEM_..."
1,724F4WGRJEXUTK6WUC2SKM6Z,Small blue,JQKHXVTRQQUJC4OWF5F6HQFK,HMDBGTNWIZGZF2YU7NZJZO4X,2500,USD,"<ApiResponse [{""catalog_object"":{""type"":""ITEM_..."
2,METCIVRJ3HTZ5QQUU2YQTSTJ,Large red,JQKHXVTRQQUJC4OWF5F6HQFK,HMDBGTNWIZGZF2YU7NZJZO4X,3000,USD,"<ApiResponse [{""catalog_object"":{""type"":""ITEM_..."
3,W3G4W5LRBJ65LKTMLDM2SIB5,Large blue,JQKHXVTRQQUJC4OWF5F6HQFK,HMDBGTNWIZGZF2YU7NZJZO4X,3000,USD,"<ApiResponse [{""catalog_object"":{""type"":""ITEM_..."
4,6B7HUOAHA4T37Y4HCNBH3JRK,Blue,2S33CD4247XXEORTAT5XYOQB,G7MKUFJHREIIZWQG4AI4FUCT,1000,USD,"<ApiResponse [{""catalog_object"":{""type"":""ITEM_..."
5,5Y2WYUW246EEZURQMHX2SKI5,Red,2S33CD4247XXEORTAT5XYOQB,G7MKUFJHREIIZWQG4AI4FUCT,1000,USD,"<ApiResponse [{""catalog_object"":{""type"":""ITEM_..."
6,EMFBRVV54DDDQPVA32SUQUST,Green,THBCUS7E6THY4UVPKZJU7VSJ,G7MKUFJHREIIZWQG4AI4FUCT,700,USD,"<ApiResponse [{""catalog_object"":{""type"":""ITEM_..."
7,WG77ONG6TJ63MCVJSJT5XMFZ,Red,THBCUS7E6THY4UVPKZJU7VSJ,G7MKUFJHREIIZWQG4AI4FUCT,700,USD,"<ApiResponse [{""catalog_object"":{""type"":""ITEM_..."
8,RROX5BQWFMGYR5BV444XRDG6,Default,Z63NWS6BEBKNPDSR6AXPUSYV,G7MKUFJHREIIZWQG4AI4FUCT,1500,USD,"<ApiResponse [{""catalog_object"":{""type"":""ITEM_..."


In [26]:
# merge back
df = pd.merge(df.drop("variation_id", axis=1), 
              df_variation.drop(["result", "category_id","variation_amount","variation_currency"], axis=1), 
         on=["variation_name","item_id"], how= 'left')

In [28]:
df_update = df_original.copy()
df_update[["category_id", "item_id", "variation_id"]] = df[["category_id", "item_id", "variation_id"]]
df_update

,variation_id,variation_name,item_id,item_name,category_id,category_name,variation_amount,variation_currency,item_description
0,ZW4ME2IFXGWXZ4HU7MESVVCW,Small red,C4UFKX2NK6CRZHFY35UL3F6H,Shirt,ON6V2PIFKB4PTQA4P3HWXB4C,Clothes,2500,USD,this is a shirt item
1,KZUFUPYCDM437SZJPPLHN6MI,Small blue,C4UFKX2NK6CRZHFY35UL3F6H,Shirt,ON6V2PIFKB4PTQA4P3HWXB4C,Clothes,2500,USD,this is a shirt item
2,GQBRVUFAZLC27UMXYYDTFJ5B,Large red,C4UFKX2NK6CRZHFY35UL3F6H,Shirt,ON6V2PIFKB4PTQA4P3HWXB4C,Clothes,3000,USD,this is a shirt item
3,HI6C3M3L7QYSTI2SJ6DF5PHN,Large blue,C4UFKX2NK6CRZHFY35UL3F6H,Shirt,ON6V2PIFKB4PTQA4P3HWXB4C,Clothes,3000,USD,this is a shirt item
4,TVTJ4R5AX7XLMZSRB6VA4AVG,Blue,EZHKUVQI63JH3CVSJPBLA24L,Hat,KLIFH7YBU7RG3MMMKDSKF72U,Accessories,1000,USD,this is a hat item
5,XUTZMBKDRFFCUFZAERXFJS6E,Red,EZHKUVQI63JH3CVSJPBLA24L,Hat,KLIFH7YBU7RG3MMMKDSKF72U,Accessories,1000,USD,this is a hat item
6,M2D423IT2XLK5ZOLA2EYXLAJ,Green,EUCSFATJLVXNHGNA6ZBONTSV,Scarf,KLIFH7YBU7RG3MMMKDSKF72U,Accessories,700,USD,this is a scarf item
7,RHUCJP6KCAAROLGIQ5GOIWVE,Red,EUCSFATJLVXNHGNA6ZBONTSV,Scarf,KLIFH7YBU7RG3MMMKDSKF72U,Accessories,700,USD,this is a scarf item
8,FADFMUXSZXDLR7CPJHM3PZFG,NaN,EFORNLRPA7XUMR22SPQ5FMBV,Sunglasses,KLIFH7YBU7RG3MMMKDSKF72U,Accessories,1500,USD,NaN


In [29]:
# clear_catalog(None)

## HOME WORK
* Reason about why use Int type for variation_amount instead of float (e.g., use 2500 to represent $25.00)
* get rid of the Regular variation in 2nd layer (i.e., Name:Regular, price:Variable). (tips: using result from insert_item)
* Cleanning work: make the functions into a class (e.g., insert_category, insert_item, insert_variation, clear_catalog), refactor the hardcoded column names (or add column name checking)
* (optional: create modifer: https://developer.squareup.com/docs/catalog-api/design-a-catalog#modifiers)
* retrieve(list) catalog in a robust way (tips: using cursor) (will describe in list payment notebook)

In [30]:
# last part, retrieve(list) catalog in a robust way (tips: using cursor)